<a href="https://colab.research.google.com/github/andrenp2/FinancialStudyPython/blob/master/Invest08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08 - Classificando Empresas da Bolsa segundo a Magic Fomrula (Analise Fundamentalista)

*** veremos nessa aula como classificar as empresas por uma analise mais fundamentalista como de Benjamin Grahan por exemplo, utilizando o site Fundamentus.com.br ***


*** 8.1 - Importando Bibliotecas ***

In [3]:
import numpy as np
import pandas as pd
import string
import warnings

warnings.filterwarnings('ignore')

*** 8.2 - Importando Dados *** 

In [4]:
# criando variavel para puxar os dados do site fundamentus 

url = 'http://www.fundamentus.com.br/resultado.php'

In [7]:
# criando um data frae (df)

df = pd.read_html(url)
df

# ele pega todos os data frame do site, como queremos so a tabela dos reulstados

df = pd.read_html(url)[0]
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.0,0.00000,0.0,0.0,"0,00%",0.0,0,0.0,0,0.0,0.0,"0,00%","0,00%",0,"0,00%","12,99%",000,9.257.250.00000,0.0,"-2,71%"
1,CFLU4,1.0,0.00000,0.0,0.0,"0,00%",0.0,0,0.0,0,0.0,0.0,"8,88%","10,72%",110,"17,68%","32,15%",000,60.351.00000,6.0,"8,14%"
2,CLAN3,0.0,0.00000,0.0,0.0,"0,00%",0.0,0,0.0,0,0.0,0.0,"0,00%","0,00%",0,"0,00%","-1,05%",000,1.012.240.00000,0.0,"-63,96%"
3,CLSC6,0.0,0.00000,0.0,0.0,"0,00%",0.0,0,0.0,0,0.0,0.0,"0,00%","0,00%",0,"0,00%","22,88%",000,1.551.160.00000,0.0,"6,66%"
4,CSTB3,15000.0,0.00000,0.0,0.0,"0,00%",0.0,0,0.0,0,0.0,0.0,"40,85%","28,98%",260,"22,40%","20,11%",000,8.420.670.00000,14.0,"31,91%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,CEPE6,2000.0,1.54585,90.0,-18326.0,"3,46%",167.0,-971,7138.0,-33,30999.0,25658.0,"-25,67%","-1,19%",95,"0,27%","0,06%",43902,1.667.000.00000,336.0,"8,63%"
883,CEPE3,2453.0,1.89599,110.0,-22477.0,"0,00%",204.0,-1191,8755.0,-41,32616.0,26996.0,"-25,67%","-1,19%",95,"0,27%","0,06%",000,1.667.000.00000,336.0,"8,63%"
884,BRAP3,3249.0,2.00490,107.0,0.0,"2,24%",1025.0,0,-64741.0,0,-64888.0,-64888.0,"0,00%","0,00%",0,"0,00%","0,05%",830.41500,10.522.400.00000,0.0,"0,00%"
885,CEPE5,2700.0,2.08690,121.0,-24740.0,"2,33%",225.0,-1311,9637.0,-45,33498.0,27726.0,"-25,67%","-1,19%",95,"0,27%","0,06%",5.51744,1.667.000.00000,336.0,"8,63%"


*** 8.3 - Tratando os dados ***

vamos tratar os dados, por exemplo, no site utilizam virgulas para separar, e no python pontos. 
Colocaremos entao decimal "," e milhar "."  


In [8]:
df = pd.read_html(url, decimal=',',thousands='.')[0]
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,VNET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","12,99%",0.00,9.257250e+09,0.00,"-2,71%"
1,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.00,6.035100e+07,0.06,"8,14%"
2,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.00,1.012240e+09,0.00,"-63,96%"
3,CLSC6,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","22,88%",0.00,1.551160e+09,0.00,"6,66%"
4,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.00,8.420670e+09,0.14,"31,91%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,CEPE6,20.00,1545.85,0.90,-18.326,"3,46%",0.167,-9.71,71.38,-0.33,309.99,256.58,"-25,67%","-1,19%",0.95,"0,27%","0,06%",439.02,1.667000e+09,3.36,"8,63%"
883,CEPE3,24.53,1895.99,1.10,-22.477,"0,00%",0.204,-11.91,87.55,-0.41,326.16,269.96,"-25,67%","-1,19%",0.95,"0,27%","0,06%",0.00,1.667000e+09,3.36,"8,63%"
884,BRAP3,32.49,2004.90,1.07,0.000,"2,24%",1.025,0.00,-647.41,0.00,-648.88,-648.88,"0,00%","0,00%",0.00,"0,00%","0,05%",830415.00,1.052240e+10,0.00,"0,00%"
885,CEPE5,27.00,2086.90,1.21,-24.740,"2,33%",0.225,-13.11,96.37,-0.45,334.98,277.26,"-25,67%","-1,19%",0.95,"0,27%","0,06%",5517.44,1.667000e+09,3.36,"8,63%"


outra coisa que precisamos fazer é que os sinais de porcentagem ali estao em strings, e precisaremos passar paran numeros 

In [18]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].astype(str).str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100


df.shape
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
13,OMGE3,35.34,-9824.35,2.18,5.958,0.000000e+00,0.778,10.21,18.31,-1.53,29.78,18.15,3.253000e+14,0.48,2.46,4.730000e+00,-2.000000e-02,2.497770e+07,2.762890e+09,1.62,7.027000e+01
17,SQIA3,18.72,-730.46,3.12,7.130,0.000000e+00,2.169,4.08,2362.57,6.82,1898.73,59.99,3.000000e-02,-0.97,6.81,2.100000e-01,-4.300000e-01,1.797330e+07,4.232550e+08,0.19,2.446000e+01
22,TUPY3,19.39,-309.69,1.16,0.562,0.000000e+00,0.443,1.65,6.12,-6.35,8.32,4.81,9.180000e+11,-0.18,1.95,1.060000e+00,-3.800000e-01,1.573130e+07,2.403220e+09,0.99,1.213000e+01
24,COGN3,6.55,-291.85,0.67,1.803,5.100000e-01,0.339,3.88,16.38,-1.07,28.68,11.40,1.101000e+01,-0.67,1.97,2.300000e-01,-2.300000e-01,3.531060e+08,1.829470e+10,0.66,7.530000e+00
27,BKBR3,10.33,-232.15,1.36,0.826,4.900000e-01,0.687,29.66,48.32,-2.41,52.91,7.27,1.710000e+00,-0.36,1.12,1.720000e+00,-5.800000e-01,2.450540e+07,1.735260e+09,0.40,2.328000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,STBP3,5.55,331.67,2.77,3.836,3.200000e-01,1.162,12.41,63.10,-2.83,64.18,19.18,6.080000e+00,1.16,2.22,2.130000e+00,8.400000e-01,2.393050e+07,1.339860e+09,0.33,3.240000e+14
875,LINX3,24.00,359.58,2.60,5.544,4.700000e+13,1.815,6.90,30.85,16.11,27.94,14.76,1.797000e+01,1.54,2.72,8.440000e+13,7.200000e-01,5.269270e+07,1.745060e+09,0.18,1.578000e+01
877,LWSA3,42.63,371.23,8.41,13.154,0.000000e+00,4.913,11.21,141.93,17.77,132.36,53.64,9.270000e+11,3.54,2.73,7.320000e+00,2.270000e+00,3.571760e+07,6.344790e+08,0.31,0.000000e+00
878,VALE3,55.63,400.02,1.64,1.974,2.540000e+00,0.669,6.76,4.77,-1.80,5.36,4.29,4.137000e+12,-1.02,1.72,1.698000e+01,4.100000e+14,1.505490e+09,1.788540e+11,0.55,1.438000e+01


*** 8.4 - Analisando os Dados ***

vamos fazer um filtro para pegar empresas com liquidez

por exemplo quero que a liquidez seja maior que 1mi, por dia durante 2 meses. Entao vamos tirar as empresas pequenas, com poucas liquidez.

In [17]:
df = df[df['Liq.2meses'] > 1000000 ]
df.shape

(205, 21)

Listar as empresas que tem o menor relacao EV/EBIT e maior ROIC que eh utilziado por joel na formula magica. 

*** O EV/EBIT entao, significa: VALOR DA EMPRESA PRECIFICADA PELO MERCADO ***

EV = ( Preco x Num de acoes emitidos ) + dividas liquidas 
EBIT = ( Lucro antes de juros e impostos descontados) 

*** Quanto MENOR o EV/EBIT, MELHOR!!! ***

ja o ROIC:

*** ROIC : ***




In [21]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values



ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,GGBR4
1,2,SMLS3,GGBR3
2,3,ENAT3,TOTS3
3,4,WIZS3,SMLS3
4,5,SULA11,BRKM5
...,...,...,...
145,146,SUZB3,SGPS3
146,147,PTBL3,STBP3
147,148,RADL3,RCSL4
148,149,PNVL3,BKBR3


*** Entao, segundo a FORMULA MAGICA, temos que somar os rankins de EV/EBIT e ROIC e analisar *** 

In [22]:
a = ranking.pivot_table(columns='EV/EBIT',values='pos')

b = ranking.pivot_table(columns='ROIC',values='pos')

In [23]:
t = pd.concat([a,b])
t

,AALR3,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,ARZZ3,B3SA3,BEEF3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,CAMB3,CAML3,CARD3,CCPR3,CCRO3,CEAB3,CESP6,CIEL3,CMIG3,CMIG4,COCE5,COGN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CVCB3,CYRE3,DIRR3,DTEX3,ECOR3,...,SGPS3,SHUL4,SLCE3,SMLS3,SMTO3,SULA11,SUZB3,TASA4,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMP3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VIVT3,VIVT4,VLID3,VULC3,VVAR3,WIZS3,WSON33,YDUQ3,BKBR3,BTOW3,CNTO3,EZTC3,HAPV3,LWSA3,RCSL4,STBP3,WEGE3
pos,112.0,89.0,69.0,143.0,76.0,8.0,48.0,117.0,139.0,136.0,27.0,107.0,46.0,134.0,67.0,116.0,44.0,104.0,79.0,72.0,73.0,80.0,106.0,28.0,13.0,11.0,55.0,138.0,51.0,21.0,23.0,63.0,125.0,26.0,41.0,7.0,127.0,60.0,131.0,56.0,...,109.0,86.0,47.0,2.0,87.0,5.0,146.0,103.0,36.0,81.0,34.0,66.0,64.0,50.0,78.0,77.0,24.0,30.0,110.0,32.0,61.0,57.0,6.0,74.0,75.0,33.0,43.0,140.0,4.0,17.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,138.0,31.0,15.0,58.0,130.0,12.0,84.0,25.0,67.0,71.0,52.0,110.0,95.0,5.0,NaN,7.0,37.0,106.0,79.0,26.0,76.0,116.0,133.0,72.0,91.0,92.0,108.0,NaN,13.0,40.0,41.0,83.0,24.0,33.0,88.0,32.0,NaN,124.0,127.0,68.0,...,146.0,105.0,96.0,4.0,104.0,NaN,141.0,103.0,63.0,136.0,145.0,62.0,61.0,98.0,3.0,85.0,36.0,NaN,117.0,NaN,128.0,129.0,57.0,115.0,114.0,19.0,42.0,119.0,48.0,66.0,56.0,149.0,144.0,150.0,121.0,43.0,111.0,148.0,147.0,9.0


In [24]:
rank = t.dropna(axis=1).sum()
rank

AALR3     250.0
ABEV3     120.0
AGRO3      84.0
ALPA4     201.0
ALSO3     206.0
          ...  
VULC3      85.0
VVAR3     259.0
WIZS3      52.0
WSON33     83.0
YDUQ3     138.0
Length: 140, dtype: float64

In [25]:
# para ordenar entao

rank.sort_values()[:15]

SMLS3      6.0
ALUP11    20.0
EQTL3     25.0
MRFG3     29.0
LEVE3     30.0
CVCB3     39.0
PSSA3     50.0
WIZS3     52.0
VLID3     52.0
ENBR3     57.0
CSMG3     59.0
TRPL4     60.0
CPLE3     61.0
SAPR4     62.0
VALE3     63.0
dtype: float64

*** Obs: podemos remover empresas do setor financeiro, como bancos pois nao faz muito sentido analisar roic etc... ***


*** Obs2: Entao o rank mostra as empresas que estao "BARATAS" porem tem potencial de trazer um grande retorno. ***

*** Tudo isso foi desenvolvido a partir da Formula Magica, nao sendo uma recomendacao de carteira, e sim uma ferramenta auxiliar para a analise *** 